<a href="https://colab.research.google.com/github/mon-project-repos/ReviewGenerator/blob/master/ReviewGenerator_FitGeneratorNextWord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminaries

## GPU Session

In [0]:
# set up session 
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


import os
import tensorflow as tf
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

## Imports

In [0]:
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
import re


import matplotlib.image  as mpimg
#import matplotlib.pyplot as plt
#%matplotlib ipympl
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])

import nltk
# nltk.download('stopwords')
#nltk.download('punkt')

from tensorflow.keras.preprocessing.sequence import pad_sequences

## Data

In [0]:
percent_validation = 0.2


In [0]:
dta = pickle.load(open("gdrive/My Drive/dta_review.pkl",'rb'))
dta = dta.loc[dta['prod_total']>100]
first_validate = int(len(dta)*(1-percent_validation))
print('Total',dta.shape, 'first validate:' ,first_validate)

review_train = dta['review_data'].iloc[:first_validate]
review_validate = dta['review_data'].iloc[first_validate: ]
#corpus_train = (" end-of-review ").join( )
#corpus_validation = (" end-of-review ").join( dta['review_data'].iloc[first_validate:])
dta = None

# NLP preprocessing

## User and Product Documents

In [0]:


#remove_filters='"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n' # don't remove anything as they may be emojis
max_vocab_words = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_vocab_words+1,
                                                  lower=True, 
                                                  filters="",
                                                 oov_token="<OOV>",
                                                  char_level=False)


corpus_train = (" endOfReview/n").join(review_train)
tokenizer.fit_on_texts(tf.keras.preprocessing.text.text_to_word_sequence(corpus_train))
total_words = len(tokenizer.word_index) + 1

word_freq = tokenizer.word_counts
word_idx = tokenizer.word_index
print(word_idx)
print(word_freq)

#print(len(tokenizer.word_index))
print(len(word_freq))

idx_word = {v:k for k,v in word_idx.items()}
print(idx_word)
print(len(idx_word))

In [10]:
print(review_train[0])
print( tokenizer.texts_to_sequences([review_train[0]])[0])

I have tried all different styles of Asics for running shoes, Contend, Cumulus, Nimbus, Scram,  GT-1000, and GT-2000. All have been great shoes with the exception of the Scram which are technically trail runners but they were just to narrow in the toe box. These are a bit stiffer than other Asics but I hope that means they will hold up. They don't have as much cushion as the Cumulus or Nimbus and I will be going back to one of those for running. These are still good shoes, I just prefer the others.Edit:  These were bothering my feet while running. I bought gel inserts and they are so much better! The original inserts have no cushioning at all. 
[3, 19, 224, 43, 182, 762, 17, 99, 11, 69, 1, 1, 1, 1, 1, 1, 5, 1, 43, 19, 104, 32, 8, 28, 4, 1649, 17, 4, 5926, 139, 13, 4899, 689, 1048, 18, 9, 45, 49, 7, 133, 15, 4, 122, 1, 14, 13, 6, 130, 1650, 66, 96, 99, 18, 3, 480, 33, 1305, 9, 53, 446, 1, 9, 128, 19, 34, 108, 269, 34, 4, 1846, 73, 648, 5, 3, 53, 44, 222, 110, 7, 112, 17, 425, 11, 1, 14,

## Create input sequences

In [0]:
#len_ngram = 20

def get_sequences(reviews,  len_ngram=20):
    sequences = []
    review_sentences = " endOfReview/n".join(reviews)
    review_tokens = tokenizer.texts_to_sequences([review_sentences])[0] #map each word to word_index 
    #for review in review_sentences:
        #review_words = review + " endOfReview/n"
        #tokenized_review = tokenizer.texts_to_sequences([review_words])[0] #map each word to word_index 
        #print(len(tokenized_review))
    #for i in range(1, min( [maxlen_ngram,len(review_words)])): # make ngrams
    for i in range(len_ngram, len(review_tokens)):
        sequences.append( review_tokens[i-len_ngram:i] )
    #if max_review_len is None:
    #    max_review_len = max([len(x) for x in sequences])
    #print(max_review_len)
    #sequences = np.array(pad_sequences(sequences,
    #                                     maxlen=maxlen_ngram,
    #                                     padding='pre')) 
    return np.array(sequences) #, max_review_len
        
def get_pred_lastword(sequences):
    predictors, last_word = sequences[:,:-1] , sequences[:,-1]
    #last_word = tf.keras.utils.to_categorical(last_word, num_classes=max_vocab_words+1)
    return (predictors, last_word )

        


In [0]:
# get max length 
len_ngram = 20
train_sequences = get_sequences(review_train,len_ngram=len_ngram)
pred_train, lastword_train = get_pred_lastword(train_sequences)

validate_sequences = get_sequences(review_validate, len_ngram=len_ngram)
pred_validate, lastword_validate = get_pred_lastword(validate_sequences)


In [13]:
print('x=', pred_train.shape, 'y=', lastword_train.shape)
print(pred_train[:3])
print(lastword_train[:3])


x= (1667162, 19) y= (1667162,)
[[  3  19 224  43 182 762  17  99  11  69   1   1   1   1   1   1   5   1
   43]
 [ 19 224  43 182 762  17  99  11  69   1   1   1   1   1   1   5   1  43
   19]
 [224  43 182 762  17  99  11  69   1   1   1   1   1   1   5   1  43  19
  104]]
[ 19 104  32]


# LSTM Model

In [14]:
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128), 
#                                         input_shape=(maxlen_ngram, max_vocab_words)))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(max_vocab_words))
# model.add(tf.keras.layers.Activation('softmax'))


print(pred_train.shape)
print(lastword_train.shape)
for m in range(24):
    print((" ").join([idx_word[i] if i>0 else "" for i in train_sequences[m]]))

print(review_train[0])

(1667162, 19)
(1667162,)
i have tried all different styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have
have tried all different styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been
tried all different styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great
all different styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great shoes
different styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great shoes with
styles of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great shoes with the
of asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great shoes with the exception
asics for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been great shoes with the exception of
for running <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> and <OOV> all have been grea

In [0]:
def generator(pred_list, next_word_list, batch_size):
    index = 0
    while True:
        last_idx = min([ index+batch_size, len(pred_list)])
        x = pred_list[index:last_idx,:]
        y = tf.keras.utils.to_categorical(next_word_list[index:last_idx], num_classes = max_vocab_words+1)
        index = index+batch_size if index+batch_size<len(pred_list) else 0
        yield x, y
        
        

In [0]:
#print(model.summary())

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_vocab_words +1 , len_ngram, input_length=len_ngram-1))
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)))
model.add(tf.keras.layers.LSTM(32, return_sequences=True))
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dropout(0.2))
#model.add(tf.keras.layers.Dense(10, activation='relu'))#, kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(max_vocab_words +1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


In [0]:
class earlyStopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('acc')> 0.9):
            print("\n Training Accuracy is 0.9.  Stopping here")
            self.model.stop_training = True
         
callbacks = earlyStopCallback()

In [23]:
batch_size = 2048
history = model.fit_generator(generator(pred_train, lastword_train, batch_size),
                   steps_per_epoch = int(len(pred_train)/batch_size)+1,
                   epochs=3,
                   callbacks=[callbacks],
                   validation_data=generator(pred_validate, lastword_validate, batch_size),
                   validation_steps = int(len(pred_validate)/batch_size)+1)

Epoch 1/3
815/815 [==============================] - 748s 918ms/step - loss: 5.7762 - acc: 0.1686 - val_loss: 5.3880 - val_acc: 0.1618
Epoch 2/3
815/815 [==============================] - 741s 910ms/step - loss: 5.3180 - acc: 0.1718 - val_loss: 5.3852 - val_acc: 0.1618
Epoch 3/3
815/815 [==============================] - 741s 909ms/step - loss: 5.3145 - acc: 0.1718 - val_loss: 5.3846 - val_acc: 0.1618


In [0]:


#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r-.')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r-.')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()

In [52]:
seed_text = "These shoes"
next_words = 10

for m in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=len_ngram-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    if predicted[0] in idx_word:
        output_word = idx_word[predicted[0]]
    seed_text += output_word
    
print(seed_text)

        

These shoes<OOV><OOV><OOV><OOV><OOV><OOV><OOV><OOV><OOV><OOV>
